In [2]:
import requests
import re 
import time
from bs4 import BeautifulSoup 
from fake_useragent import UserAgent
import pandas as pd
import numpy as np
from random import randint
import selenium.webdriver

In [3]:
city_url = 'https://raw.githubusercontent.com/ucsb-dreamlab/dianpingBusinessScrape/master/City_URL.csv'
city = pd.read_csv(city_url, index_col= 0)
city.head(10)

,city_name,URL
0,阿拉善,www.dianping.com/alashan/ch75/g34311
1,鞍山,www.dianping.com/anshan/ch75/g34311
2,安庆,www.dianping.com/anqing/ch75/g34311
3,安阳,www.dianping.com/anyang/ch75/g34311
4,阿坝,www.dianping.com/aba/ch75/g34311
5,安顺,www.dianping.com/anshun/ch75/g34311
6,阿里,www.dianping.com/ali/ch75/g34311
7,安康,www.dianping.com/ankang/ch75/g34311
8,安国市,www.dianping.com/anguo/ch75/g34311
9,安新县,www.dianping.com/anxin/ch75/g34311


Filter this dataset by city levels. Now it has 411 results, and most cities in this filtered dataset are tier 1-3 city. 

In [4]:
filtered_city = city[-(
                city['city_name'].str.endswith('市')| 
                city['city_name'].str.endswith('县')|
                city['city_name'].str.endswith('镇')| 
                city['city_name'].str.endswith('旗')|
                city['city_name'].str.endswith('村')| 
                city['city_name'].str.endswith('区'))].reset_index(drop=True)
display(filtered_city)

,city_name,URL
0,阿拉善,www.dianping.com/alashan/ch75/g34311
1,鞍山,www.dianping.com/anshan/ch75/g34311
2,安庆,www.dianping.com/anqing/ch75/g34311
3,安阳,www.dianping.com/anyang/ch75/g34311
4,阿坝,www.dianping.com/aba/ch75/g34311
...,...,...
406,诸暨,www.dianping.com/zhuji/ch75/g34311
407,章丘,www.dianping.com/zhangqiu/ch75/g34311
408,周庄,www.dianping.com/zhouzhuang/ch75/g34311
409,朱家尖,www.dianping.com/zhujiajian/ch75/g34311


In [39]:
from selenium.webdriver import ChromeOptions

In [124]:
!pip install mitmproxy

1. https://stackoverflow.com/questions/33225947/can-a-website-detect-when-you-are-using-selenium-with-chromedriver
2. https://stackoverflow.com/questions/55501524/how-does-recaptcha-3-know-im-using-selenium-chromedriver/55502835#55502835
3. https://stackoverflow.com/questions/53039551/selenium-webdriver-modifying-navigator-webdriver-flag-to-prevent-selenium-detec



In [40]:
def getHtmlContent(url): 
 UA = UserAgent(verify_ssl = False)  
 #Retrieve cookies with selenium
 options = ChromeOptions()
 #options.add_argument("--disable-blink-features=AutomationControlled")
 options.add_experimental_option("excludeSwitches", ['enable-automation'])
 driver = selenium.webdriver.Chrome(options=options)
 
 #driver_option.add_argument("--headless")
 #print(driver.execute_script("return navigator.userAgent;"))
 print("Scraping from "+ url)
 driver.get(url)
 all_cookies = driver.get_cookies()
 cookies_dict = {}
 for cookies in all_cookies:
   cookies_dict[cookies['name']] = cookies['value']
 
 #Set User Agent in request headers
 
 headers = {
    "User-Agent": UA.random
    }
 #Request HTML with session
 try:
   s = requests.Session()
   #r = s.get(url, headers = headers, cookies = {'from-my':'browser'})
   r = s.get(url, headers = headers, cookies= cookies_dict)
   r.raise_for_status()
   if(r.encoding != "UTF-8"):
      r.encoding = r.apparent_encoding
   return r.text
 except:
   return "Request Error"

In [41]:
URL =  "http://"+ filtered_city['URL'][55] # add "https" to avoid schema error 
URL

'http://www.dianping.com/dongguan/ch75/g34311'

In [46]:
HTML = getHtmlContent(URL)

Scraping from http://www.dianping.com/dongguan/ch75/g34311


In [47]:
soup = BeautifulSoup(HTML, "html.parser")
print(soup)

<html>
<head>
<link href="//www.dpfile.com/app/pc-common/dp_favicon.a4af753914321c8e82e402e2b4be01d7.ico" rel="icon" type="image/x-icon"/>
<link href="//www.dpfile.com/app/pc-common/dp_favicon.a4af753914321c8e82e402e2b4be01d7.ico" rel="shortcut icon" type="image/x-icon"/>
<title>东莞绘本馆机构_课程_价格_排名-大众点评网</title>
<meta content="东莞绘本馆,东莞绘本馆机构,东莞绘本馆课程,东莞绘本馆价格,东莞绘本馆机构排名,大众点评网" name="Keywords"/>
<meta content="大众点评网学习培训频道为您找到最新最全的东莞绘本馆机构。点击查看更多关于东莞绘本馆机构课程、价格、评价、排名、电话、地址等信息。" name="Description"/>
<meta content="format=html5; url=https://m.dianping.com/dongguan/ch75/g34311" http-equiv="mobile-agent"/>
<link href="https://m.dianping.com/dongguan/ch75/g34311" media="only screen and (max-width: 640px)" rel="alternate"/>
<meta charset="utf-8">
<script type="text/javascript">
        window._DP_HeaderData = {
            'cityId': '219',                   
            'cityChName': '东莞',                 
            'cityEnName': 'dongguan',               
            'pageType': 'search',           

In [48]:
SubURL = []
LibName = []
for div in soup.find_all(class_ = "tit"):
    for a in div.find_all('a', href = True):
        SubURL.append(a.get('href'))
        LibName.append(a.getText().strip())

//*[@id="shop-all-list"]/ul/li[1]/div[2]/div[1]/a/h4

In [49]:
city_lib =pd.DataFrame({'Library_Name':LibName,'URL': SubURL}) 
shop_lib = city_lib[city_lib['URL'].str.contains("shop")].reset_index(drop = True)
shop_lib

,Library_Name,URL
0,博沃思·专注力·心理咨询·家庭教育(东城中心),https://www.dianping.com/shop/l9aKfsT5ai99979f
1,博沃思注意力心理咨询(南城民盈国贸校区),https://www.dianping.com/shop/l3rGpmYP8Pmi2Y7c
2,巧虎KIDS成长中心(南城旺南奥特莱斯校区),https://www.dianping.com/shop/kafJm0OSb07Rb0QP
3,棒吉拉国际早教(东城世博校区),https://www.dianping.com/shop/k8O23jycCAlNs1BD
4,棒吉拉国际早教(南城联景校区),https://www.dianping.com/shop/k2cncDNeqiZbMhzy
5,棒吉拉国际早教中心(金域华府店),https://www.dianping.com/shop/l36ITBGXABzNElMs
6,上语书田(莞城店),https://www.dianping.com/shop/G7gsOAQ43tvVJmtC
7,书鲸阅读海洋第一国际校区,https://www.dianping.com/shop/i7Ylg8PjQrAWZeR1
8,橘子绘本馆,https://www.dianping.com/shop/kaue1mlrFmTq7Nas
9,海洋国际双语阅读馆,https://www.dianping.com/shop/H88gFr4Ts770cgQu


In [50]:
subpage_soup = []
for i in range(0,len(shop_lib)):
    subpage_soup.append(BeautifulSoup(getHtmlContent(url = shop_lib['URL'][i]), "html.parser"))
    time.sleep(randint(1,2))

Scraping from https://www.dianping.com/shop/l9aKfsT5ai99979f
Scraping from https://www.dianping.com/shop/l3rGpmYP8Pmi2Y7c
Scraping from https://www.dianping.com/shop/kafJm0OSb07Rb0QP
Scraping from https://www.dianping.com/shop/k8O23jycCAlNs1BD
Scraping from https://www.dianping.com/shop/k2cncDNeqiZbMhzy
Scraping from https://www.dianping.com/shop/l36ITBGXABzNElMs
Scraping from https://www.dianping.com/shop/G7gsOAQ43tvVJmtC
Scraping from https://www.dianping.com/shop/i7Ylg8PjQrAWZeR1
Scraping from https://www.dianping.com/shop/kaue1mlrFmTq7Nas
Scraping from https://www.dianping.com/shop/H88gFr4Ts770cgQu
Scraping from https://www.dianping.com/shop/k6MKgQpmRZWbOq5C
Scraping from https://www.dianping.com/shop/k2WSnqVHtsiJQmvS
Scraping from https://www.dianping.com/shop/i4rmxT0rd4UE3Uf7
Scraping from https://www.dianping.com/shop/G3KfqebaT0b2q48e
Scraping from https://www.dianping.com/shop/H9fwqb0whByd7iPC


In [27]:
#subpage_soup[7]

In [51]:
AddressList = []
HourList = []
PhoneNumList = []


In [52]:
for soup in subpage_soup:
    div = soup.find_all(class_ = "address")
    AddressList.append(div[0].find_all(text=True, recursive=False)[1].strip())

In [53]:
AddressList

['东泰社区鸿福东路1号民盈国贸城5号楼',
 '万达广场万达百货',
 '莞太路旺南奥特莱斯3楼303室',
 '南城区西平联景大厦3楼',
 '世博广场D区2楼',
 '香园路33号摩天艺术中心2楼',
 '东纵大道8号3楼',
 '东莞市南城区第一国际财富中心博学城二层',
 '凤岗镇昌盛南路35号四楼',
 '体育路都汇大厦602',
 '未来世界花园169号',
 '莲峰北路沃多夫风临公馆3楼',
 '龙城一路汇星商业中心5栋1606室',
 '东昇路石井综合楼2号楼3楼',
 '东纵路莞城段2号3004室']

In [54]:
for soup in subpage_soup:
    hour_tag = soup.find(class_ = "mod shop-info")
    if hour_tag is not None:
       for li in hour_tag.find_all('li'):
         for span in li.find_all('span'): 
            match = span.text 
            if match == "营业时间":
                str = li.find_all(text=True, recursive=False)[1].strip().replace("\n", " ")
                HourList.append(str) 
    else:
         HourList.append(np.NaN)


In [55]:
HourList

['周一至周日 10:00-20:00',
 '周一至周日 10:00-20:00',
 '周一,周三至周日 09:00-18:00',
 '周一至周日 08:00-18:00',
 '周一至周日 08:00-20:00',
 '周一至周日 08:30-12:00 14:30-18:30',
 '周一至周日 10:00-22:00',
 '周六,周日 08:30-18:30  周三至周五 09:00-21:00',
 '周二至周五 14:00-20:00  周六,周日 10:00-18:30',
 '周六,周日 08:00-21:00  周一至周五 14:00-21:00',
 '周二至周日 09:30-18:00',
 '周一至周日 08:00-18:00',
 '周一至周日 10:00-22:00',
 '周二至周日 09:00-21:00',
 '周一至周日 10:00-22:00']

In [56]:
for soup in subpage_soup: 
    phone_tag = soup.find(class_ = "phone")
    if phone_tag is not None:
        phonenum = phone_tag.find('span')
        PhoneNumList.append(phonenum['data-phone'])
    else: 
        PhoneNumList.append(np.NaN)

In [57]:
PhoneNumList

['15907552646',
 '15907552646',
 '4000303096-04603',
 '4000303096-87514',
 '4000303096-26351',
 '4000303096-27917',
 '4000303096-00006',
 '18929227556',
 '15820888755',
 '4000303096-36220',
 '13377700884',
 '16507690768',
 '0769-81823888',
 '13922931039',
 '0769-21661849']

In [58]:
shop_lib['Address'] = AddressList
shop_lib['Business_Hour'] = HourList
shop_lib['Phone_Number'] = PhoneNumList
shop_lib

,Library_Name,URL,Address,Business_Hour,Phone_Number
0,博沃思·专注力·心理咨询·家庭教育(东城中心),https://www.dianping.com/shop/l9aKfsT5ai99979f,东泰社区鸿福东路1号民盈国贸城5号楼,周一至周日 10:00-20:00,15907552646
1,博沃思注意力心理咨询(南城民盈国贸校区),https://www.dianping.com/shop/l3rGpmYP8Pmi2Y7c,万达广场万达百货,周一至周日 10:00-20:00,15907552646
2,巧虎KIDS成长中心(南城旺南奥特莱斯校区),https://www.dianping.com/shop/kafJm0OSb07Rb0QP,莞太路旺南奥特莱斯3楼303室,"周一,周三至周日 09:00-18:00",4000303096-04603
3,棒吉拉国际早教(东城世博校区),https://www.dianping.com/shop/k8O23jycCAlNs1BD,南城区西平联景大厦3楼,周一至周日 08:00-18:00,4000303096-87514
4,棒吉拉国际早教(南城联景校区),https://www.dianping.com/shop/k2cncDNeqiZbMhzy,世博广场D区2楼,周一至周日 08:00-20:00,4000303096-26351
5,棒吉拉国际早教中心(金域华府店),https://www.dianping.com/shop/l36ITBGXABzNElMs,香园路33号摩天艺术中心2楼,周一至周日 08:30-12:00 14:30-18:30,4000303096-27917
6,上语书田(莞城店),https://www.dianping.com/shop/G7gsOAQ43tvVJmtC,东纵大道8号3楼,周一至周日 10:00-22:00,4000303096-00006
7,书鲸阅读海洋第一国际校区,https://www.dianping.com/shop/i7Ylg8PjQrAWZeR1,东莞市南城区第一国际财富中心博学城二层,"周六,周日 08:30-18:30 周三至周五 09:00-21:00",18929227556
8,橘子绘本馆,https://www.dianping.com/shop/kaue1mlrFmTq7Nas,凤岗镇昌盛南路35号四楼,"周二至周五 14:00-20:00 周六,周日 10:00-18:30",15820888755
9,海洋国际双语阅读馆,https://www.dianping.com/shop/H88gFr4Ts770cgQu,体育路都汇大厦602,"周六,周日 08:00-21:00 周一至周五 14:00-21:00",4000303096-36220


In [106]:
#Summarized the above code into a function
def parse_info_citylib(URL):
    HTML = getHtmlContent(URL)
    soup = BeautifulSoup(HTML, "html.parser")
    SubURL = []
    LibName = []
    for div in soup.find_all(class_ = "tit"):
        for a in div.find_all('a', href = True):
            SubURL.append(a.get('href'))
            LibName.append(a.getText().strip())
    city_lib =pd.DataFrame({'Library_Name':LibName,'URL': SubURL}) 
    shop_lib = city_lib[city_lib['URL'].str.contains("shop")].reset_index(drop = True)




    subpage_soup = []
    for i in range(0,len(shop_lib)): #Need to change this part
        subpage_soup.append(BeautifulSoup(getHtmlContent(url = URL), "html.parser"))
        time.sleep(randint(1,2))

    AddressList = []
    HourList = []
    PhoneNumList = []

    for soup in subpage_soup:
        div = soup.find_all(class_ = "address")
        AddressList.append(div[0].find_all(text=True, recursive=False)[1].strip())
    
    for soup in subpage_soup:
        hour_tag = soup.find(class_ = "mod shop-info")
        if hour_tag is not None:
            for li in hour_tag.find_all('li'):
                for span in li.find_all('span'): 
                    match = span.text 
                    if match == "营业时间":
                        str = li.find_all(text=True, recursive=False)[1].strip().replace("\n", " ")
                        HourList.append(str) 
        else:
            HourList.append(np.NaN)
    
    for soup in subpage_soup:
        phone_tag = soup.find(class_ = "phone")
        if phone_tag is not None:
            phonenum = phone_tag.find('span')
            PhoneNumList.append(phonenum['data-phone'])
        else: 
            PhoneNumList.append(np.NaN)
            
    shop_lib['Address'] = AddressList
    shop_lib['Business_Hour'] = HourList
    shop_lib['Phone_Number'] = PhoneNumList


    return shop_lib

In [60]:
Second_URL="http://"+ filtered_city['URL'][1]
Second_URL

'http://www.dianping.com/anshan/ch75/g34311'

In [50]:
lib_info = parse_info_citylib(Second_URL)

[nan, nan, nan, nan, nan, nan]